# RSI momentum trading strategy example

- This is a backtest example notebook
    - The notebook picks up the best parameters from the grid search results from notebook v5

# Set up

Set up Trading Strategy data client.


In [1]:
from tradeexecutor.utils.notebook import setup_charting_and_output
from tradingstrategy.client import Client

client = Client.create_jupyter_client()

setup_charting_and_output()  # Set chart and table output formatting options

Started Trading Strategy in Jupyter notebook environment, configuration is stored in /Users/moo/.tradingstrategy


# Load data

We use Binance data so we get a longer period of data.

In [2]:
import datetime
from tradingstrategy.timebucket import TimeBucket
from tradeexecutor.utils.binance import create_binance_universe

strategy_universe = create_binance_universe(
    ["BTCUSDT", "ETHUSDT"],   # Binance internal tickers later mapped to Trading strategy DEXPair metadata class
    candle_time_bucket=TimeBucket.d1,
    stop_loss_time_bucket=TimeBucket.h4,
    start_at=datetime.datetime(2019, 1, 1),  # Backtest for 5 years data
    end_at=datetime.datetime(2024, 1, 1),
    include_lending=False
)


  0%|          | 0/2 [00:00<?, ?it/s]

# Show loaded trading universe

Display generic troubleshooting information about the loaded data.

In [3]:
pairs = strategy_universe.data_universe.pairs  # Trading pairs metadata
candles = strategy_universe.data_universe.candles  # Candles for all trading pairs

print(f"Loaded {candles.get_candle_count():,} candles.")

for pair in pairs.iterate_pairs():
    pair_candles = candles.get_candles_by_pair(pair)
    first_close = pair_candles.iloc[0]["close"]
    first_close_at = pair_candles.index[0]
    print(f"Pair {pair} first close price {first_close} at {first_close_at}")

Loaded 21,914 candles.
Pair <Pair #1 BTC - USDT at exchange binance> first close price 3693.13 at 2019-01-01 00:00:00
Pair <Pair #2 ETH - USDT at exchange binance> first close price 131.32 at 2019-01-01 00:00:00


# Trading algorithm

In [4]:
from tradeexecutor.strategy.weighting import weight_equal, weight_by_1_slash_n, weight_passthrouh
from tradeexecutor.strategy.alpha_model import AlphaModel
import numpy as np
from tradeexecutor.strategy.pandas_trader.position_manager import PositionManager
from tradingstrategy.chain import ChainId
from typing import List, Dict

from pandas_ta.momentum import rsi
import pandas as pd

from tradeexecutor.strategy.trading_strategy_universe import TradingStrategyUniverse
from tradeexecutor.state.visualisation import PlotKind, PlotShape, PlotLabel
from tradeexecutor.state.trade import TradeExecution
from tradeexecutor.strategy.pricing_model import PricingModel
from tradeexecutor.state.state import State

# List of pair descriptions we used to look up pair metadata
our_pairs = [
    (ChainId.centralised_exchange, "binance", "BTC", "USDT"),
    (ChainId.centralised_exchange, "binance", "ETH", "USDT"),
]

rsi_days = 9  # The length of RSI indicator
eth_btc_rsi_days = 60  # The length of ETH/BTC RSI
rsi_high = 70  # RSI trigger threshold for decision making
rsi_low = 50  # RSI trigger threshold for decision making
allocation = 0.9 # Allocate 90% of cash to each position
lookback_candles = 120
minimum_rebalance_trade_threshold = 500.00  # Don't do trades that would have less than 500 USD value change

def decide_trades(
    timestamp: pd.Timestamp,
    strategy_universe: TradingStrategyUniverse,
    state: State,
    pricing_model: PricingModel,
    cycle_debug_data: Dict
) -> List[TradeExecution]:

    # Resolve our pair metadata for our two pair strategy
    position_manager = PositionManager(timestamp, strategy_universe, state, pricing_model)
    alpha_model = AlphaModel(timestamp)
    btc_pair = position_manager.get_trading_pair(our_pairs[0])
    eth_pair = position_manager.get_trading_pair(our_pairs[1])

    position_manager.log("decide_trades() start")

    #
    # Indicators
    #
    # Calculate indicators for each pair.
    #

    # Per-trading pair calcualted data
    close_prices = {btc_pair: None, eth_pair: None}  # Recent close prices
    current_rsi_values = {}  # RSI yesterday
    previous_rsi_values = {}  # RSI day before yesterday
    current_price = {}  # Close price yesterday
    momentum = {btc_pair: 0, eth_pair: 0}
    eth_btc_rsi_yesterday = None

    for pair in [btc_pair, eth_pair]:

        pair_candles = candles.get_last_entries_by_pair_and_timestamp(pair.internal_id, timestamp)
        close_prices[pair] = pair_candles["close"]

        assert pair_candles is not None
        rsi_series = rsi(pair_candles["close"], length=rsi_days)  # Will return None if the data buffer does not have enough days to look back

        # Reset indicators for this cycle and this pair
        current_rsi_values[pair] = None
        previous_rsi_values[pair] = None
        current_price[pair] = None
        price_today = None

        if len(pair_candles) > 0:
            # We have enough data to get the latest price
            current_price[pair] = price_today = pair_candles["close"][-1]

        if rsi_series is not None:
            current_val = rsi_series[-1]
            if np.isfinite(current_val):
                # We have enough data and good value for RSI
                assert 0 < current_val < 100, f"RSI sanity check failed: {pair}: {current_val}"  # Check we are in expected range
                current_rsi_values[pair] = current_val

            previous_val = rsi_series[-2]
            if np.isfinite(previous_val):
                # We have enough data and good value for RSI
                assert 0 < previous_val < 100, f"RSI sanity check failed: {pair}: {previous_val}"  # Check we are in expected range
                previous_rsi_values[pair] = previous_val

            previous_rsi_values[pair] = previous_val

    eth_btc_price = close_prices[eth_pair] / close_prices[btc_pair]
    eth_btc_rsi = rsi(eth_btc_price, length=eth_btc_rsi_days)
    if eth_btc_rsi is not None:
        eth_btc_rsi_yesterday = eth_btc_rsi[-1]
        momentum[eth_pair] = eth_btc_rsi_yesterday / 100
        momentum[btc_pair] = 1 - momentum[eth_pair]

    #
    # Trading logic
    #

    for pair in [btc_pair, eth_pair]:

        existing_position = position_manager.get_current_position_for_pair(pair)
        pair_open = existing_position is not None
        pair_momentum = momentum.get(pair, 0)
        signal_strength = max(pair_momentum, 0.1)  # Singal strength must be positive, as we do long-only
        if pd.isna(signal_strength):
            signal_strength = 0
        alpha_model.set_signal(pair, 0)

        if pair_open:
            # We have existing open position for this pair,
            # keep it open by default unless we get a trigger condition below
            position_manager.log(f"Pair {pair} already open")
            alpha_model.set_signal(pair, signal_strength)

        if current_rsi_values[pair] and previous_rsi_values[pair]:

            # Check for RSI crossing our threshold values in this cycle, compared to the previous cycle
            rsi_cross_above = current_rsi_values[pair] >= rsi_high and previous_rsi_values[btc_pair] < rsi_high
            rsi_cross_below = current_rsi_values[pair] < rsi_low and previous_rsi_values[pair] > rsi_low

            if not pair_open:
                # Check for opening a position if no position is open
                if rsi_cross_above:
                    position_manager.log(f"Pair {pair} crossed above")
                    alpha_model.set_signal(pair, signal_strength)
            else:
                # We have open position, check for the closure condition
                if rsi_cross_below:
                    position_manager.log(f"Pair {pair} crossed below")
                    alpha_model.set_signal(pair, 0)

    # Equally weight for all assets that
    alpha_model.select_top_signals(2)
    alpha_model.assign_weights(weight_passthrouh)
    alpha_model.normalise_weights()
    alpha_model.update_old_weights(state.portfolio)
    portfolio = position_manager.get_current_portfolio()
    portfolio_target_value = portfolio.get_total_equity() * allocation
    alpha_model.calculate_target_positions(position_manager, portfolio_target_value)
    trades = alpha_model.generate_rebalance_trades_and_triggers(
        position_manager,
        min_trade_threshold=minimum_rebalance_trade_threshold,
    )

    #
    # Visualisations
    #

    visualisation = state.visualisation  # Helper class to visualise strategy output

    visualisation.plot_indicator(
        timestamp,
        f"ETH",
        PlotKind.technical_indicator_detached,
        current_price[eth_pair],
        colour="blue",
    )

    # Draw BTC + ETH RSI between its trigger zones for this pair of we got a valid value for RSI for this pair

    # BTC RSI daily
    visualisation.plot_indicator(
        timestamp,
        f"RSI",
        PlotKind.technical_indicator_detached,
        current_rsi_values[btc_pair],
        colour="orange",
    )

    # ETH RSI daily
    visualisation.plot_indicator(
        timestamp,
        f"RSI ETH",
        PlotKind.technical_indicator_overlay_on_detached,
        current_rsi_values[eth_pair],
        colour="blue",
        label=PlotLabel.hidden,
        detached_overlay_name=f"RSI",
    )

    # Low (vertical line)
    visualisation.plot_indicator(
        timestamp,
        f"RSI low trigger",
        PlotKind.technical_indicator_overlay_on_detached,
        rsi_low,
        detached_overlay_name=f"RSI",
        plot_shape=PlotShape.horizontal_vertical,
        colour="red",
        label=PlotLabel.hidden,
    )

    # High (vertical line)
    visualisation.plot_indicator(
        timestamp,
        f"RSI high trigger",
        PlotKind.technical_indicator_overlay_on_detached,
        rsi_high,
        detached_overlay_name=f"RSI",
        plot_shape=PlotShape.horizontal_vertical,
        colour="red",
        label=PlotLabel.hidden,
    )

    if eth_btc_rsi_yesterday is not None:

        visualisation.plot_indicator(
            timestamp,
            f"ETH/BTC",
            PlotKind.technical_indicator_detached,
            eth_btc_price[-1],
            colour="grey",
        )

        if pd.notna(eth_btc_rsi_yesterday):
            visualisation.plot_indicator(
                timestamp,
                f"ETH/BTC RSI",
                PlotKind.technical_indicator_detached,
                eth_btc_rsi_yesterday,
                colour="grey",
            )

    state.visualisation.add_calculations(timestamp, alpha_model.to_dict())  # Record alpha model thinking

    return trades

# Backtest

In [5]:
import logging
from tradeexecutor.strategy.cycle import CycleDuration
from tradeexecutor.backtest.backtest_runner import run_backtest_inline

state, universe, debug_dump = run_backtest_inline(
    name="RSI multipair",
    engine_version="0.3",
    decide_trades=decide_trades,
    client=client,
    cycle_duration=CycleDuration.cycle_1d,
    universe=strategy_universe,
    initial_deposit=10_000,
    strategy_logging=False,
)

trade_count = len(list(state.portfolio.get_all_trades()))
print(f"Backtesting completed, backtested strategy made {trade_count} trades")

  0%|          | 0/157838400 [00:00<?, ?it/s]

Backtesting completed, backtested strategy made 852 trades


# Benchmark

Benchmark against

- Buy and hold BTC
- Buy and hold ETH

In [8]:
from tradeexecutor.visual.benchmark import visualise_benchmark

btc_pair = strategy_universe.data_universe.pairs.get_pair_by_human_description(our_pairs[0])
eth_pair = strategy_universe.data_universe.pairs.get_pair_by_human_description(our_pairs[1])

benchmark_indexes = pd.DataFrame({
    "BTC": strategy_universe.data_universe.candles.get_candles_by_pair(btc_pair)["close"],
    "ETH": strategy_universe.data_universe.candles.get_candles_by_pair(eth_pair)["close"],
})
benchmark_indexes["BTC"].attrs = {"colour": "orange"}
benchmark_indexes["ETH"].attrs = {"colour": "blue"}

fig = visualise_benchmark(
    name=state.name,
    portfolio_statistics=state.stats.portfolio,
    all_cash=state.portfolio.get_initial_deposit(),
    benchmark_indexes=benchmark_indexes,
    height=800
)

fig.show()

# Performance metrics

Compare popular portfolio performance metrics.

- Benchmark the strategy against buy and hold ETH portfolio

In [9]:
from tradeexecutor.visual.equity_curve import calculate_equity_curve, calculate_returns, generate_buy_and_hold_returns
from tradeexecutor.analysis.advanced_metrics import visualise_advanced_metrics, AdvancedMetricsMode

equity = calculate_equity_curve(state)
returns = calculate_returns(equity)
benchmark_returns = generate_buy_and_hold_returns(benchmark_indexes["ETH"])

metrics = visualise_advanced_metrics(
    returns,
    mode=AdvancedMetricsMode.full,
    benchmark=benchmark_returns,
)

display(metrics)

,Strategy,Benchmark
Start Period,2019-01-01,2019-01-01
End Period,2024-01-01,2024-01-01
Risk-Free Rate,0.0%,0.0%
Time in Market,60.0%,100.0%
Cumulative Return,"5,496.45%","1,631.50%"
CAGR﹪,123.56%,76.83%
Sharpe,1.86,1.1
Prob. Sharpe Ratio,100.0%,99.29%
Smart Sharpe,1.73,1.02
Sortino,3.03,1.63


# Trading metrics

Trading-related metrics like fees.



In [10]:
from tradeexecutor.analysis.trade_analyser import build_trade_analysis

analysis = build_trade_analysis(state.portfolio)
summary = analysis.calculate_summary_statistics()
display(summary.to_dataframe())

,0
Trading period length,1791 days 0 hours
Return %,5496.45%
Annualised return %,1120.16%
Cash at start,"$10,000.00"
Value at end,"$559,644.71"
Trade volume,"$22,948,205.95"
Position win percent,60.53%
Total positions,76
Won positions,46
Lost positions,30
